<a href="https://colab.research.google.com/github/ppunkz/CE888_DS/blob/master/Lab%205%20/%20Recommender%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

#  Load the data from ``jester-data-1.csv''


In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [18]:
arr = df.values
print(arr.shape)

(24983, 101)


In [19]:
rated = np.where(arr!=99)
rated

(array([    0,     0,     0, ..., 24982, 24982, 24982]),
 array([ 0,  1,  2, ..., 70, 71, 87]))

In [20]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1835357 (1835357,)


# Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [23]:
arr[idx[0][0], idx[1][0]]

5.68

In [24]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 2

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)


In [0]:
sgd(15)

24.554625455698442
24.500366192158708


In [31]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.45842842e+01,  7.63114891e-01,  5.97104636e-01, ...,
         9.32557860e-01,  5.96119801e-02,  1.52282272e+00],
       [ 9.65821782e+01,  1.15879806e+00,  6.59829257e-02, ...,
         8.15115612e-01,  3.15628808e-02,  1.11415823e+00],
       [ 5.47247782e+01,  6.80590962e-01, -6.21014579e-02, ...,
         4.06642811e-01,  1.14649088e-02,  5.10627263e-01],
       ...,
       [ 3.79392074e+01,  3.74588584e-01,  3.60064716e-01, ...,
         5.05631982e-01,  3.39577481e-02,  8.42948414e-01],
       [ 3.77835773e+01,  5.10860562e-01, -2.12671889e-01, ...,
         1.86527112e-01, -3.03959426e-03,  1.46606180e-01],
       [ 7.43134215e+01,  7.57553005e-01,  6.06503799e-01, ...,
         9.35590813e-01,  6.01418422e-02,  1.53132284e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [33]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 74.58428421284829)","(-7.82, 0.7631148907048166)","(8.79, 0.5971046358905193)","(-9.66, 0.5659839696725155)","(-8.16, -0.8802849427707863)","(-7.52, 0.7502540951146915)","(-8.5, 1.8063955933992455)","(-9.85, -0.12759323786608262)","(4.17, -0.4016214516161512)","(-8.98, -0.30124057856262354)","(-4.76, 1.2841838496027016)","(-8.5, 2.0343876803248833)","(-6.75, 1.827712601518673)","(-7.18, -1.409835377581001)","(8.45, 1.8554693196072625)","(-7.18, -1.5148402807144647)","(-7.52, -2.97354650685873)","(-7.43, -0.801787811090763)","(-9.81, -0.6325228155633479)","(-9.85, 0.2388535311245169)","(-9.85, -0.7522149010558427)","(-9.37, 2.5550947835059357)","(1.5, 1.2359709931220721)","(-4.37, 0.3800770003431179)","(-9.81, -1.2810819663617854)","(-8.5, 0.7832898459272187)","(1.12, 1.6931605283446436)","(7.82, 3.4233767530756727)","(2.86, 1.6891147178352612)","(9.13, 3.2085523334847026)","(-7.43, -0.07755326568631951)","(2.14, 2.490833268203728)","(-4.08, 3.214183500347153)","(-9.08, -0.8160512301794054)","(7.82, 1.27531319767324)","(5.05, 3.1222878858340417)","(4.95, 3.4654102140929566)","(-9.17, -0.92926343470607)","(-8.4, 1.437254089811006)","(-8.4, 1.516812096763598)",...,"(8.59, 2.7527904133276166)","(3.59, 3.0308119324164635)","(-6.84, 0.5071628395649743)","(-9.03, -0.17255973601173835)","(2.82, 2.688599204371541)","(-1.36, 2.7033974222912285)","(-9.08, -0.5970983444012978)","(8.3, 2.745500177588041)","(5.68, 2.877059810453832)","(-4.81, 0.9022607099340313)","(99.0, -0.29859572047096333)","(99.0, 2.354235020189475)","(99.0, 0.8374643073312014)","(99.0, -1.006713950352009)","(99.0, -0.08790586424769088)","(99.0, 1.9373719103000064)","(99.0, 0.5129545768070766)","(-9.42, 1.308538406192862)","(99.0, 0.22269247711822654)","(99.0, 0.9610772157543672)","(99.0, 1.6562352404610665)","(-7.72, 0.980364440896964)","(99.0, 2.010804991523453)","(99.0, 0.7730225749234436)","(99.0, 0.9787596104779023)","(99.0, 0.3147496224171373)","(99.0, 1.5333795001517536)","(99.0, 1.6523261550513468)","(99.0, 2.82548609908041)","(99.0, 0.7939593434464894)","(2.82, 1.6574658854172657)","(99.0, 1.0251991725322296)","(99.0, 2.1790645593867577)","(99.0, 0.9327459525881002)","(99.0, 0.7862395351825622)","(99.0, 1.251495363480553)","(-5.63, 1.3909444313046817)","(99.0, 0.9325578596138734)","(99.0, 0.05961198008122364)","(99.0, 1.5228227234226708)"
1,"(100.0, 96.58217823618847)","(4.08, 1.1587980627363883)","(-0.29, 0.06598292571392415)","(6.36, 0.20574415868524748)","(4.37, -1.7711938006746575)","(-2.38, 0.27960598764277544)","(-9.66, 1.4716803611495641)","(-0.73, -0.603536972943795)","(-5.34, -0.7429305635205056)","(8.88, -0.7923389862046888)","(9.22, 1.4234333680390687)","(6.75, 1.8428844219063503)","(8.64, 1.4155184931089073)","(4.42, -2.3833028231259754)","(7.43, 1.2669919281271314)","(4.56, -2.1978954922415124)","(-0.97, -3.6947852038215037)","(4.66, -1.47891908347945)","(-0.68, -0.7430275724022768)","(3.3, 0.17962196889318188)","(-1.21, -1.321244817455198)","(0.87, 2.3024509852957413)","(8.64, 0.7758859039822271)","(8.35, -0.026676817297042906)","(9.17, -1.964491131646602)","(0.05, 0.1764591777440698)","(7.57, 1.1867883807846278)","(4.71, 3.639017289649168)","(0.87, 1.6321716257923735)","(-0.39, 3.4660442166283)","(6.99, -0.758041597817327)","(6.5, 2.481523121477325)","(-0.92, 3.7751432643696043)","(7.14, -1.7442079419292262)","(9.03, 0.7983919474505862)","(-1.8, 3.5837421915795176)","(0.73, 3.866284528691329)","(7.09, -1.6852425367775234)","(3.4, 1.401060176384258)","(-0.87, 0.9805941815173272)",...,"(-6.7, 2.7433577071420743)","(-3.35, 3.56009569834688)","(-9.03, 0.040889235712382906)","(4.47, -0.9708126178138234)","(4.08, 2.536552735702007)","(-3.83, 2.9454782934221897)","(8.74, -0.9991167733741717)","(1.12, 3.094720787